In [2]:
#import standard libraries
import os
from typing import List
import json


In [3]:
#import Langchain + Langsmith libraries
from langchain_community.document_loaders import TextLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain.vectorstores import Pinecone
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate, ChatPromptTemplate, HumanMessagePromptTemplate
from langchain_core.messages import SystemMessage
from langchain.memory import ConversationBufferWindowMemory
from langchain.tools.retriever import create_retriever_tool
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.tools import tool
from langsmith import Client
from langchain_core.output_parsers import StrOutputParser
from langchain.agents import AgentExecutor
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field, validator
from langchain.memory import ConversationBufferMemory
from langchain_core.messages.human import HumanMessage
from langchain.prompts import PromptTemplate

from langchain_experimental.text_splitter import SemanticChunker
from langchain_pinecone import PineconeVectorStore
from langchain_openai import ChatOpenAI
from langchain.agents import create_react_agent

c:\Windows\System32\knowledgenow2\env\Lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [4]:
#Import utils
from utils.airtable import AirTable

In [49]:
#import secrets
import dotenv

config = dotenv.dotenv_values(".env")
OPENAI_API_KEY= config['OPENAI_API_KEY']
PINECONE_API_KEY= config['PINECONE_API_KEY']
COHERE_API_KEY = config["COHERE_API_KEY"]


In [7]:
#Set up LangSmith
from uuid import uuid4

unique_id = uuid4().hex[0:8]
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = f"Tracing Walkthrough - {unique_id}"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = "ls__c4c7a9772ef046038636017b03b5f17b"

In [8]:
langsmith_client = Client()

In [9]:
#import data and prompts 
from prompts.current.prompts_context_enrichment_agent_2024_03_26 import context_enrichment_template
from prompts.current.prompts_Subquery_agent_2024_03_26 import Subquery_template
from prompts.current.prompts_evaluator_agent_2024_03_26 import evaluator_template
from prompts.current.prompts_answer_agent_2024_03_26 import answer_template
from prompts.current.prompts_rag_extractor_agent_2024_03_26 import Rag_extractor_template

In [10]:
# import Global Context file
global_context_file_name = "prompts/current/global_context_2024_03_26.txt"

with open(global_context_file_name, 'r') as file:
    global_context =  file.read()


Context Agent 

Reading files vs pasting the text 
Reading files allows us to change the version of the file externally without needing to copy-paste the changes. Part of why having a clear file structure is so useful, you always know where things are.

In [11]:
#create context_enrichment prompt template
context_enrichment_template = context_enrichment_template

context_prompt = PromptTemplate.from_template(template=context_enrichment_template, MessagePlaceholder=["(agent_scratchpad)"])

In [12]:
#initiaze language models
model_name = "gpt-4-turbo-preview"
llm  = ChatOpenAI(
    openai_api_key=OPENAI_API_KEY, 
    model=model_name, 
    temperature= 0.0
    )

In [13]:
@tool
def Context_awareness(query:str)-> str:
    "Use this tool any time you get questions about meetings, tasks, need context about the activities of the organization Fasterise, its members or projects."
    global global_context
    return global_context

In [14]:
context_tools = [Context_awareness]

There are multiple declarations of the variable "agent", "agent_executor" and "tools" across the code. Even if this sequential flow from agent to agent was set in stone... what about 6 months in the future? What about when a Moderator Agent changes the order of which agent goes next?

Note: always a better practice to give variables their own name instead of overwriting.

In [15]:
context_enrichment_agent = create_react_agent(llm=llm, 
                           tools=context_tools, 
                           prompt=context_prompt)

In [16]:
context_enrichment_agent_executor = AgentExecutor(
    agent=context_enrichment_agent, 
    tools=context_tools, 
    verbose=True, 
    handle_parsing_errors=True, 
    )

In [17]:
human_prompt = "what did alejo and bartek discuss in the meeting"

context_enriched_output = context_enrichment_agent_executor.invoke({"input": human_prompt})



> Entering new AgentExecutor chain...


Question: what did Alejo and Bartek discuss in the meeting
Thought: To enrich this prompt, I need to understand the context of Alejo and Bartek's roles within Fasterise, the projects they are involved in, and any recent organizational goals or challenges that might influence the topics of their discussion.
Action: Context_awareness
Action Input: Alejo and Bartek meeting discussion contextGlobal Context of Fasterise

### Mission and Vision
- **Mission and Vision**: Fasterise is centered around leveraging AI technologies to amplify human potential. 

There are 4 teams at Fasterise: [Team: Members]
- **Marketing and Branding**: Paige.
- **Sales and Outreach**: Zach, Leo.
- **Programming & Development**: Alejo, Henry, Bartek, Hubi, Kuba.
- **Company Operations**: Pawel, Alejo.

### Team Structure and Roles
- **Henry**: Experience in Marketing, Go High Level CRM. Also experience with Python and building Chatbots with Voiceflow.
- **Alejo**: Plays a key role in Operations, Sales, and Develop

In [18]:
enriched_prompt = context_enriched_output["output"]
print(enriched_prompt)

Given the strategic roles of Alejo and Bartek within Fasterise, focusing on Operations, Sales, Development, and Engineering respectively, and considering the company's mission to leverage AI technologies to amplify human potential, it is highly probable that their meeting discussion centered on the technical and operational progress of current projects such as KnowledgeNow and the Personalized Outreach System. The conversation likely covered development challenges, strategic planning, and the integration of AI technologies to advance the company's innovative objectives.


SUB-QUERY GENERATOR AGENT


In [19]:
class sub_query(BaseModel):
    sub_queries: list = Field(description="""The sub queries extracted from the context_enriched_prompt"""
)
    
parser = JsonOutputParser(pydantic_object=sub_query)


In [20]:
@tool
def context_awareness_enriched_tools(rag_query:str)-> str:
    "Use this tool any time you need to access the context-enriched prompt"
    global enriched_prompt
    return enriched_prompt

In [21]:
tools = [context_awareness_enriched_tools]

In [22]:
Subqueries_template = Subquery_template

Subquery_prompt = PromptTemplate.from_template(template=Subqueries_template, MessagePlaceholder=["(agent_scratchpad)"])

In [23]:
Subquery_agent = create_react_agent(llm=llm, 
                                    tools = tools, 
                                    prompt=Subquery_prompt)

In [24]:
Subquery_agent_executor = AgentExecutor(agent=Subquery_agent, tools=tools, verbose=True,handle_parsing_errors=True)

In [25]:

rag_sub_queries = Subquery_agent_executor.invoke({"human_prompt":enriched_prompt,
                       "chat_history": [], 
                       "format_instructions":parser.get_format_instructions()})
print(rag_sub_queries)



> Entering new AgentExecutor chain...


```json
{
  "sub_queries": [
    "What are the latest technical advancements in the KnowledgeNow project?",
    "What operational challenges are currently being faced in the KnowledgeNow project?",
    "How is AI technology being integrated into the KnowledgeNow project?",
    "What are the recent development milestones for the Personalized Outreach System?",
    "What operational issues are impacting the Personalized Outreach System?",
    "How is AI technology being utilized in the Personalized Outreach System?",
    "What strategic planning discussions have Alejo and Bartek had regarding the use of AI to amplify human potential?",
    "What are the roles of Alejo and Bartek in the strategic development of Fasterise's projects?"
  ]
}
```Invalid Format: Missing 'Action:' after 'Thought:```json
{
  "sub_queries": [
    "What are the latest technical advancements in the KnowledgeNow project?",
    "What operational challenges are currently being faced in the KnowledgeNow project?",
   

In [26]:
rag_query = rag_sub_queries["output"]

# Assuming rag_query contains the JSON string as shown in your output
# rag_query = '```json\n{\n  "sub_queries": [\n    "List all topics discussed by Alejo and Bartek in the meeting.",\n    "Identify any action items assigned to Alejo and Bartek during the meeting.",\n    "Extract any decisions made by Alejo and Bartek in the meeting.",\n    "Summarize the key points Alejo and Bartek agreed upon during the meeting."\n  ]\n}\n```'

# Remove the markdown code block syntax (```json and ```) from the string
rag_query_cleaned = rag_query.replace('```json\n', '').replace('\n```', '')

# Convert the JSON string into a dictionary
rag_query_dict = json.loads(rag_query_cleaned)

# Extract the sub-queries using a list comprehension
sub_queries = [sub_query for sub_query in rag_query_dict["sub_queries"]]

# Display the extracted sub-queries
for i, sub_query in enumerate(sub_queries, start=1):
    print(f"sub_query_{i}: {sub_query}")


sub_query_1: What are the latest technical advancements in the KnowledgeNow project?
sub_query_2: What operational challenges are currently being faced in the KnowledgeNow project?
sub_query_3: How is AI technology being integrated into the KnowledgeNow project?
sub_query_4: What are the recent development milestones for the Personalized Outreach System?
sub_query_5: What operational issues are impacting the Personalized Outreach System?
sub_query_6: How is AI technology being utilized in the Personalized Outreach System?
sub_query_7: What strategic planning discussions have Alejo and Bartek had regarding the use of AI to amplify human potential?
sub_query_8: What are the roles of Alejo and Bartek in the strategic development of Fasterise's projects?


MULTIQUERY

In [27]:
#loading company docs
def load_file(data):
    loader = DirectoryLoader("data",
                    glob="**/*.txt",
                    loader_cls= TextLoader)
    text_documents = loader.load()
    return text_documents
     


In [28]:
#extract the data 
extracted_data = load_file(["2024-03-12_Dev_Meeting_-_Knowledge_Now_summary_1.txt, 2024-03-12 Dev Meeting - Knowledge Now Final Draft 1cleaned.txt"])
print(extracted_data)

[Document(page_content='', metadata={'source': 'data\\raw\\discord_chats.txt'}), Document(page_content="Speaker 4\nAloha, Henry. I was just talking to my dad and I was basically saying about the Zoom knowledge base thing and he brought up a good point, was Microsoft Copilot. It can do that a lot with a lot of the teams, all the teams calls. \n\nSpeaker 2\nSorry, do what? \n\nSpeaker 4\nSo Microsoft Copilot can ask it, like who contributes the most on the calls about profit sharing. That's one of the features of Microsoft Copilot. So Copilot already has this knowledge base. So that is a competitor to think about. So then do that. And to what extent do Microsoft have that? So that's something we should probably look into is copilot. \n\nSpeaker 2\nGood call. Very good call. Yeah, we have access to the full Microsoft ecosystem, including code pilot through the founders hub. \n\nSpeaker 4\nKnow, we could probably move everything to teams then. But do we want to? Because right now we're doi

In [29]:
#split documents into chunks
def text_split(extracted_data):
    text_splitter = SemanticChunker(OpenAIEmbeddings(api_key=OPENAI_API_KEY), breakpoint_threshold_type="percentile")
    documents = text_splitter.split_documents(extracted_data)

    return documents


In [30]:
text_chunk = text_split(extracted_data)
print("The length of the chunk:", len(text_chunk))

c:\Windows\System32\knowledgenow2\env\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.0.9 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


The length of the chunk: 937


In [33]:
#creating embeddings

def data_embedding():
    embeddings = OpenAIEmbeddings(api_key=OPENAI_API_KEY)
    return embeddings



In [34]:
embeddings = data_embedding()

In [40]:
Pinecone_api_key = PINECONE_API_KEY
Your_env = "gcp-starter"
index_name = "knowledgenow"

In [52]:

#Adding embeddings into the vectorstore
vectorstore = PineconeVectorStore.from_documents(text_chunk, embeddings, index_name=index_name )
#initializing retrival
retriver = vectorstore.as_retriever(search_kwargs={'k':25})
multi_llm = ChatOpenAI(temperature= 0.5, openai_api_key=OPENAI_API_KEY, model_name="gpt-4-turbo-preview")


retriever = MultiQueryRetriever.from_llm(
    retriever=retriver, llm=multi_llm
)

In [53]:
import logging 

logging.basicConfig()
logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)

In [54]:
#retrival chain
qa = RetrievalQAWithSourcesChain.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=retriever
)


KNOWLEDGEBASE QUERY AGENT

In [55]:
#creating a custom tool
@tool
def Knowledgebase(query:str)-> str:
    """
    Use this tool when you want extract information from the Knowledgenow database"""
    result_str = qa.invoke(sub_query)
    return result_str



In [56]:
tools = [Knowledgebase]

In [57]:
#Knowledgebase_system prompt

Rag_extractor_template = Rag_extractor_template
Rag_extractor_prompt = PromptTemplate.from_template(template=Rag_extractor_template, MessagePlaceholder=["(agent_scratchpad)"])


In [58]:
Knowledgebase_agent = create_react_agent(llm=llm, tools=tools, prompt=Rag_extractor_prompt)
Knowledgebase_agent_executor = AgentExecutor(agent=Knowledgebase_agent, tools=tools, verbose=True,handle_parsing_errors=True, max_iterations=8)


In [59]:
combined_output_list = []
for sub_query in sub_queries:
    Knowledgebase_agent_output = Knowledgebase_agent_executor.invoke({"sub_query": sub_query})
    combined_output_list.append(Knowledgebase_agent_output["output"])
    print("sub_query:", sub_query, "\n", "Knowledgebase_agent_output:", Knowledgebase_agent_output["output"], '\n###\n')



> Entering new AgentExecutor chain...
Question: What are the latest technical advancements in the KnowledgeNow project?
Thought: To answer this question, I need to query the KnowledgeNow database for the most recent updates or advancements related to the technical aspects of the KnowledgeNow project. This will involve looking for any new features, improvements, or innovations that have been documented.
Action: Knowledgebase
Action Input: What are the latest technical advancements in the KnowledgeNow project?

INFO:langchain.retrievers.multi_query:Generated queries: ['1. What are the most recent technological developments in the KnowledgeNow initiative?', '2. Can you list the new tech updates made to the KnowledgeNow project recently?', '3. What innovative technologies have been introduced in the KnowledgeNow project lately?']


{'question': 'What are the latest technical advancements in the KnowledgeNow project?', 'answer': "The latest technical advancements in the KnowledgeNow project include the development of a knowledge base system that aims to significantly enhance productivity and streamline workflows. This system, referred to as KnowledgeNow, is designed to act as a partner to each user, helping to recall tasks discussed in calls, thereby eliminating the need to repeatedly ask for such information. It is intended to integrate seamlessly with existing systems, allowing for an upgrade in productivity tools, such as converting text messages to Google Calendar entries. The project also explores the potential of using KnowledgeNow for more advanced applications, including acting as a pair programmer by understanding the objectives discussed in planning sessions and assisting in coding tasks.\n\nFurther advancements include the development of a context enrichment agent that enriches prompts with relevant inf

INFO:langchain.retrievers.multi_query:Generated queries: ['1. What are the most recent updates in technology for the KnowledgeNow initiative?', '2. Can you provide information on the newest tech developments within the KnowledgeNow project?', "3. What are the latest innovations in the KnowledgeNow project's technology?"]


{'question': 'What are the latest technical advancements in the KnowledgeNow project?', 'answer': 'The latest technical advancements in the KnowledgeNow project include the development of a knowledge base that aims to significantly enhance productivity by providing instant access to relevant information and tasks discussed in meetings. This system, referred to as KnowledgeNow, is designed to become an integral partner in streamlining workflows and facilitating the upgrade of customer systems. It features capabilities for integrating with existing systems to help users level up their operations. Additionally, the project envisions incorporating a pair of programming functionality, leveraging the knowledge base to assist in coding tasks by understanding the objectives discussed in planning sessions. The system is also focused on improving content creation, idea generation, and task management by generating clear task lists for team members post-meeting. Furthermore, the project is explor

...

{'question': 'What strategic planning discussions have Alejo and Bartek had regarding the use of AI to amplify human potential?', 'answer': "Alejo and Bartek have engaged in strategic planning discussions focusing on the use of AI to amplify human potential in various contexts. Their discussions have spanned from educational programs aimed at balancing technology in our lives to more technical aspects of AI integration in business processes and project management. They have explored the potential of AI in enhancing public speaking skills, facilitating youth programs, and even in the realm of real estate and strategic consulting. A significant part of their strategy involves leveraging AI to improve project management efficiency, automate outreach, and enhance the user experience with AI-driven assistants. Their approach is multifaceted, aiming to harness AI's power to support human activities, ensure ethical considerations, and foster innovation across different sectors.\n\n", 'sources

INFO:langchain.retrievers.multi_query:Generated queries: ['1. How have Alejo and Bartek strategized about leveraging AI to enhance human capabilities?', '2. In what ways have Alejo and Bartek discussed the strategic implementation of AI to boost human potential?', '3. What conversations have Alejo and Bartek had about planning the integration of AI to increase human capacity?']


{'question': 'What strategic planning discussions have Alejo and Bartek had regarding the use of AI to amplify human potential?', 'answer': 'Alejo and Bartek have engaged in strategic planning discussions regarding the use of AI to amplify human potential, focusing on several key areas:\n\n1. **Educational Initiatives and Workshops**: They have explored the integration of AI in educational settings, aiming to balance technology use with human interaction and learning. This includes facilitating workshops that are technology-free to encourage face-to-face interaction, while also recognizing the potential for tech-oriented foundations to support educational programs through donations of technology like iPads or computers for AI-related projects (', 'sources': 'data\\processed\\transcripts\\2024-03-01 Paige, Bartek, Pawel - Plans and Updates for Real Estatecleaned.txt).'}Final Answer: Alejo and Bartek have engaged in strategic planning discussions focusing on the use of AI to amplify huma

INFO:langchain.retrievers.multi_query:Generated queries: ['1. How do Alejo and Bartek contribute to the strategic planning of projects at Fasterise?', "2. In what capacities do Alejo and Bartek participate in shaping the strategic direction of Fasterise's projects?", '3. What responsibilities do Alejo and Bartek hold in the strategic advancement of projects within Fasterise?']


{'question': "What are the roles of Alejo and Bartek in the strategic development of Fasterise's projects?", 'answer': "Alejo and Bartek play significant roles in the strategic development of Fasterise's projects. Alejo is involved in discussions about strategic alignment and differentiation of AI technologies for the company. He focuses on developing frameworks and syllabuses for training and workshops that teach clients how to integrate AI into their workflows, emphasizing the unique knowledge and competitive advantage of each company. This includes creating a knowledge base that can be utilized to maintain a company's strategic differentiation through AI. Alejo's work is described as focusing on the strategic alignment of underlying AI and knowledge bases to ensure companies retain their competitive edge.\n\nBartek, on the other hand, is heavily involved in the technical and operational aspects of project development. He is recognized for his great work on sales and client projects,

In [62]:
print(combined_output_list)

["The latest technical advancements in the KnowledgeNow project include the development of a knowledge base system designed to enhance productivity and streamline workflows significantly. This system, referred to as KnowledgeNow, aims to become an integral partner to users by recalling tasks discussed in calls, thereby eliminating the need to repeatedly ask for such information. It is intended to integrate seamlessly with existing systems, allowing for an upgrade in productivity tools, such as converting text messages to Google Calendar entries. The project also explores the potential of using KnowledgeNow for more advanced applications, including acting as a pair programmer by understanding the objectives discussed in planning sessions and assisting in coding tasks.\n\nFurther advancements include the development of a context enrichment agent that enriches prompts with relevant information from a global context knowledge base, optimizing them for retrieval and query answering. The sys

In [63]:
print("Final Results in Sequence:")
for index, item in enumerate(combined_output_list, start=1):
    print(f"{index}. {item}\n")


Final Results in Sequence:
1. The latest technical advancements in the KnowledgeNow project include the development of a knowledge base system designed to enhance productivity and streamline workflows significantly. This system, referred to as KnowledgeNow, aims to become an integral partner to users by recalling tasks discussed in calls, thereby eliminating the need to repeatedly ask for such information. It is intended to integrate seamlessly with existing systems, allowing for an upgrade in productivity tools, such as converting text messages to Google Calendar entries. The project also explores the potential of using KnowledgeNow for more advanced applications, including acting as a pair programmer by understanding the objectives discussed in planning sessions and assisting in coding tasks.

Further advancements include the development of a context enrichment agent that enriches prompts with relevant information from a global context knowledge base, optimizing them for retrieval an

Evaluator Agent

In [ ]:
#creating a custom tool
@tool
def Vectormatches(query:str)-> str:
    """
    Use this tool when you want to return vector match results for the vectordatabase"""
    global combined_output_list
    return combined_output_list

: 

In [ ]:
#evaluator_template declared at the beginning of notebook
evaluator_template = evaluator_template
eval_prompt = PromptTemplate.from_template(template=evaluator_template, MessagePlaceholder=["(agent_scratchpad)"])

eval_agent = create_react_agent(llm=llm, tools=tools, prompt=eval_prompt)
evaluator_agent_executor = AgentExecutor(agent=eval_agent, tools=tools, verbose=True,handle_parsing_errors=True, )

: 

In [ ]:
Evalutor_output=evaluator_agent_executor.invoke({
        "input":input, 
        "global_context": global_context, 
        "enriched_prompt": enriched_prompt,
        "Vector_matches": combined_output_list})

Evaluator_agent_output=Evalutor_output["output"]
print(Evaluator_agent_output)

: 

ANSWER AGENT

In [ ]:
#creating a custom tool
@tool
def Evaluator_result(query:str)-> str:
    """
    Use this tool anytime you need to return  Re-ranked and evaluated data results from the Evaluator assistant """
    global Evaluator_agent_output
    return Evaluator_agent_output

: 

In [ ]:
tools = [Evaluator_result]

: 

In [ ]:
answer_template = answer_template

: 

In [ ]:
answer_prompt = PromptTemplate.from_template(template=answer_template, MessagePlaceholder=["(agent_scratchpad)"])
answer_agent = create_react_agent(llm=llm, tools=tools, prompt=answer_prompt)
answer_agent_executor = AgentExecutor(agent=answer_agent, tools=tools, verbose=True,handle_parsing_errors=True)

: 

In [ ]:
answer_output=answer_agent_executor.invoke({
        "input":input, 
        "global_context": global_context, 
        "enriched_prompt": enriched_prompt})

answer_agent_output = answer_output["output"]
print(answer_agent_output)

: 

In [ ]:
#conversational memory 
conversational_memory = ConversationBufferWindowMemory(
    memory_key="chat_history",
    k=5,
    return_messages=True

)

: 

In [ ]:
conversational_memory.chat_memory.add_user_message(human_prompt)
conversational_memory.chat_memory.add_ai_message(answer_agent_output)

: 

In [ ]:
conversational_memory.chat_memory.messages

: 

In [ ]:


def memory2str(memory:ConversationBufferWindowMemory):
    messages = memory.chat_memory.messages
    memory_list = [
        f"Human: {mem.content}" for mem in messages 
    ]
    memory_str = "/n".join(memory_list)
    return memory_str

: 

In [ ]:
print(memory2str(conversational_memory))

: 

### MAKE SURE TO MODIFY VARIABLES BEFORE SENDING TO AIRTABLE

In [ ]:
#LAST STEP: Post data to airtable

airtable = AirTable()

data_for_airtable = {
                "name_of_dev": "Kuba",
                "model_name": "gpt-4-Turbo",
                "human_prompt": human_prompt,
                "global_context": global_context,
                "context_enrichment_agent_system_prompt": context_enrichment_template,
                "enriched_prompt_output": enriched_prompt_output,
                "rag_extractor_agent_system_prompt": Rag_extractor_template,
                "rag_optimized_queries_output": combined_output_list,
                "total_chunks_retrieved": total_chunks_retrieved,
                "chunk_size": "937",
                "knowledge_base_docs_list": extracted_data,
                "evaluator_agent_system_prompt": evaluator_template,
                "reranked_rag_queries_output": Evaluator_agent_output,
                "answer_agent_system_prompt": answer_template,
                "final_answer_to_user": answer_agent_output
              }

print(data_for_airtable)

airtable.post_data_to_airtable(data_for_airtable)

: 

In [ ]:
def chat(text:str):
    out = answer_agent_executor.invoke({
        "input":text, 
        "chat_history": [memory2str(conversational_memory)], 
        "global_context": global_context})
    conversational_memory.chat_memory.add_user_message(text)
    conversational_memory.chat_memory.add_ai_message(out["output"])
    return out["output"]

: 

In [ ]:
while True:
    user_input=input(f"Input_Prompt:")
    result= qa({"query": user_input})
    print("Response:", result["result"])

: 